Задача поиск похожих по эмбеддингам

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

In [1]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-06-09 03:08:34--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-09 03:08:35--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc36cc26ce61358784b12055577f.dl.dropboxusercontent.com/cd/0/inline/Bm0JG7MQrWhGLP7LsCutcpsEuzpi0n6HRfTtulKhs7M4ZcNxTqbpp_NCScUeSz4NQOJDLe1FKZ5pXDEG80zcdNAe1gQ-bYlVIYFrlltDMEcEUJuLwHR_D_XOIIZ2dIyYuV1WJknfEYFABG0o6dGYMwPHwYPGtnJrKkyma0mYRfkTxA/file# [following]
--2022-06-09 03:08:35--  https://uc36cc26ce61358784b12055577f.dl.dropboxusercontent.com/cd/0/inline/Bm0JG7MQrWhGLP7LsCutcpsEuzpi0n6HRfTtulKhs7M4ZcNxTqbpp_NCScUeSz4NQOJD

#Что надо сделать
1. объединить в одну выборку

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [3]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
df = positive.append(negative)

In [4]:
df.tail()

,text
111918,Но не каждый хочет что то исправлять:( http://...
111919,скучаю так :-( только @taaannyaaa вправляет мо...
111920,"Вот и в школу, в говно это идти уже надо("
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :..."
111922,Такси везет меня на работу. Раздумываю приплат...


2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)

Сначала хочу поиграться с русскоязычными векторайзерами

In [5]:
import gensim.downloader as api

api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [6]:
import gensim.downloader as api

word_vectors = api.load('word2vec-ruscorpora-300')  # загрузим предтренированные вектора слов из gensim-data

[==================================================] 100.0% 198.8/198.8MB downloaded


In [7]:
# выведим слово наиболее близкое к 'женщина', 'король' и далекое от 'мужчина'
result = word_vectors.most_similar(positive=['женщина_NOUN', 'король_NOUN'], negative=['мужчина_NOUN'])
print("{}: {:.4f}".format(*result[0]))

королева_NOUN: 0.7314


In [8]:
# определим схожесть между словами
similarity = word_vectors.similarity('кот_NOUN', 'собака_NOUN')
print(similarity)

similarity = word_vectors.similarity('водка_NOUN', 'коньяк_NOUN')
print(similarity)

similarity = word_vectors.similarity('кот_NOUN', 'коньяк_NOUN')
print(similarity)

0.55678993
0.78255457
0.17411722


In [9]:
# найдем top-3 самых близких слов
result = word_vectors.similar_by_word("гонка_NOUN", topn=3)
print(result)

result = word_vectors.similar_by_word("акула_NOUN", topn=3)
print(result)

result = word_vectors.similar_by_word("жимолость_NOUN", topn=3)
print(result)

[('гонки_NOUN', 0.7330766916275024), ('гонщик_NOUN', 0.6996685266494751), ('брэндс::хэтч_NOUN', 0.6618280410766602)]
[('барракуда_NOUN', 0.635097086429596), ('групер_NOUN', 0.618218183517456), ('меч-рыба_NOUN', 0.6097476482391357)]
[('магония_NOUN', 0.6694105863571167), ('пузыреплодник_NOUN', 0.6618051528930664), ('актинидия_NOUN', 0.6587378978729248)]


Теперь начинаем выполнять задание

In [10]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 8.2 MB 16.0 MB/s 


In [11]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=b300b201b8f4148c8df308fa9921c88e1a754f862d3d7c7b106e8c5e28e89b9a
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [12]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 646 kB 7.9 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391590 sha256=0add45a7809051ad93029142e0ff9e1411650435bff21bf37e811844c4ad5c1e
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


## Построим модели W2V и Fasttext

In [13]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [14]:
# функция препроцессинга строки
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [15]:
# Готовим данные для обучения моделей
assert True

# Preprocess for models fitting

tweets = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

for tweet in df['text']:
    spls = preprocess_txt(tweet)
    tweets.append(spls)
    c += 1
    if c > 100000:
        break

In [16]:
tweets[8]

['поприветствовать', 'новый', 'читатель', 'alexey1789']

In [17]:
tweets = [i for i in tweets if len(i) > 2]

Обучаем модели

In [18]:
modelW2V = Word2Vec(sentences=tweets, size=300, min_count=6, window=5, workers=8)

In [19]:
modelFT = FastText(sentences=tweets, size=300, min_count=6, window=5, workers=8)

Создаем annoy твитов для модели *v2w*

In [20]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
#ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for tweet in df['text']:
    n_w2v = 0
    #n_ft = 0
    index_map[counter] = tweet
    tweet = tweet.split()

    vector_w2v = np.zeros(300)
    #vector_ft = np.zeros(300)
    for word in tweet:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        #if word in modelFT.wv:
            #vector_ft += modelFT.wv[word]
            #n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    #if n_ft > 0:
        #vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    #ft_index.add_item(counter, vector_ft)
            
    counter += 1
        
    if counter > 100000:
        break

w2v_index.build(20)
#ft_index.build(10)

True

Создадим функцию для поиска 5 ближайших твитов

In [21]:
def get_closest_tweets(tweet, index, vectorizer, index_map):
    tweet = preprocess_txt(tweet)
    vector = np.zeros(300)
    norm = 0
    for word in tweet:
        if word in vectorizer:
            vector += vectorizer[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    closest_tweets = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in closest_tweets]

Тестируем поиск ближайших твитов на модели v2w

In [22]:
TEXT = "У кого сегодня такое же хорошее настроение, как у меня?"

In [23]:
get_closest_tweets(TEXT, w2v_index, modelW2V.wv, index_map)

['@ReGiNa__007 а почему бы хорошему человеку не поднять настроение :*',
 'Хорошее настроение с самого утра обеспечено.Он репостнул мою запись)',
 'RT @SergPizza: Новогоднее настроение ) http://t.co/daZkpSwXka',
 'На улице как в сказке)))) \nХоть это настроение поднимает!',
 '@borisov_vasiliy @NikolinaTheOne , Вася, твить больше %) поднимай настроение %)']

Создаем annoy твитов для модели fasttext

In [24]:
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for tweet in df['text']:
    n_ft = 0
    index_map[counter] = tweet
    tweet = tweet.split()

    vector_ft = np.zeros(300)
    for word in tweet:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft

    ft_index.add_item(counter, vector_ft)
            
    counter += 1
        
    if counter > 100000:
        break

ft_index.build(20)

True

In [25]:
get_closest_tweets(TEXT, ft_index, modelFT.wv, index_map)

['Я человек-поднимашка-настрояшка!\nХа-ха-ха!Всем настроение поднимаю)',
 'Отличный вечер и поднятое настроение. спасибо, @Olkooooo )',
 '@shrimp_Jane ОН НЕ СТАРЕЕТ МАТЬ ЕГО...МАГИЯ ВНЕ ХОГВАРТСА ДЖА АЯ ЯЯЯЯЙ_)',
 'Одни джентельмены,как же приятно:) настроение прям подняли)',
 '#BbBbBfoo Добрый вечер фолловеры,как настроение?) #BbBo']

Русскоязычные glove не так просто найти. Нашел из в библиотеке navec

In [26]:
!pip install navec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2022-06-09 03:19:31--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  14.4MB/s    in 4.5s    

2022-06-09 03:19:36 (11.3 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [28]:
from navec import Navec

path = '/content/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)


Создаем annoy твитов для модели glove

In [29]:
gl_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for tweet in df['text']:
    n_gl = 0
    index_map[counter] = tweet
    tweet = tweet.split()

    vector_glove = np.zeros(300)
    for word in tweet:
        if word in navec.vocab:
            vector_glove += navec[word]
            n_gl += 1
    if n_gl > 0:
        vector_glove = vector_glove / n_gl

    gl_index.add_item(counter, vector_glove)
            
    counter += 1
        
    if counter > 100000:
        break

gl_index.build(20)

True

In [30]:
get_closest_tweets(TEXT, gl_index, navec, index_map)

['Самый лучший день ! ))) Первый хороший снег нападал)))))',
 'Насмеяялись:)\nСпасибо за этот хороший вечер @angelika_3829 и родителям.',
 'Как снег пошол так и новогоднее настроение появилось)))))',
 'RT @vladkabulavka: @jjanuaryj это самый классный комплимент) спасибо большое)',
 'Всем привет!Скоро Новый год:*\nЭто очень классный праздник\n#люблюего']

## Насчет слоя Embedding я не очень понял, как реализовать решение этой задачи, т.к. слой Embedding может быть только частью нейоронной сети (слоем). На чем его обучать такую модель, чтобы на выходе у нее был вектор твита я тоже не понял. Прошу прислать мне правильное решение данного задания для самостоятельного разбора. Заранее благодарю

3. Проверить насколько хорошо работают подходы

По моему ощущению на данной задаче они работают примерно одинаково )